In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
sales_dataframe = spark.read.option('header', 'True').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',')


In [5]:
sales_dataframe.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.
sales_dataframe.createOrReplaceTempView('home_sales')


In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
(SELECT
  date_built AS year,
  ROUND(AVG(price), 2) AS avgerage_price
FROM home_sales
WHERE bedrooms=4
GROUP BY date_built
ORDER BY date_built)
"""
spark.sql(query).show()


+----+--------------+
|year|avgerage_price|
+----+--------------+
|2010|     296800.75|
|2011|      302141.9|
|2012|     298233.42|
|2013|     299999.39|
|2014|     299073.89|
|2015|     307908.86|
|2016|     296050.24|
|2017|     296576.69|
+----+--------------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query = """
(SELECT
  date_built AS year,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms=3 AND bathrooms=3
GROUP BY date_built
ORDER BY date_built)
"""

spark.sql(query).show()

+----+-------------+
|year|average_price|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
(SELECT
  date_built AS year,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE
    bedrooms=3 AND bathrooms=3 AND
    floors=2 AND sqft_lot>=2000
GROUP BY date_built
ORDER BY date_built)
"""

spark.sql(query).show()


+----+-------------+
|year|average_price|
+----+-------------+
|2010|    280447.23|
|2011|    281413.45|
|2012|    295858.68|
|2013|    295142.13|
|2014|    294195.13|
|2015|    291788.36|
|2016|     287812.6|
|2017|    282026.59|
+----+-------------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
query = """
(SELECT
  view,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE price>=350000
GROUP BY view)
"""

start_time = time.time()
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.6243124008178711 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
query = """
(SELECT
view,
ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE price>=350000
GROUP BY view)
"""

start_time = time.time()
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.3570735454559326 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sales_dataframe.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

Py4JJavaError: An error occurred while calling o61.parquet.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputComm

In [33]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet('home_sales_partitioned')

In [34]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('home_sales_p')

In [35]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
query = """
(SELECT
view,
ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE price>=350000
GROUP BY view)
"""

start_time = time.time()
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.7398931980133057 seconds ---


In [36]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [37]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False